In [1]:
import json
import numpy as np
import os

In [2]:
def align_source_to_target(source,target,scale = False):

	tar = target.copy()
	sou = source.copy()
	center_tar = tar - np.mean(tar,0) # centralized target mesh
	center_sou = sou - np.mean(sou,0) # centralized source mesh

	W = np.matmul(center_tar.transpose(),center_sou)
	U,S,V = np.linalg.svd(W)
	R = np.matmul(np.matmul(V.transpose(),np.diag([1,1,np.linalg.det(np.matmul(V.transpose(),U.transpose()))])),U.transpose()) # calculate rotation matrix (exclude mirror symmetry)

	if scale:
		R_sou = np.matmul(center_sou,R)
		s = np.sum(R_sou*center_tar)/np.sum(R_sou*R_sou)
	else:
		s = 1

	t = np.mean(tar,0) - s*np.matmul(np.expand_dims(np.mean(sou,0),0),R)
	
	return R,t,s

In [3]:
def align_source_to_target_2d(source, target, scale=False):
    # 复制目标和源数据
    tar = target.copy()
    sou = source.copy()
    
    # 中心化处理
    center_tar = tar - np.mean(tar, axis=0)  # 中心化目标点集
    center_sou = sou - np.mean(sou, axis=0)  # 中心化源点集

    # 计算协方差矩阵
    W = np.matmul(center_tar.T, center_sou)  # 计算点集之间的协方差矩阵
    U, S, V = np.linalg.svd(W)  # 对协方差矩阵进行SVD分解

    # 计算旋转矩阵R (二维空间)
    R = np.matmul(V.T, U.T)  # 计算旋转矩阵

    if scale:
        # 计算缩放因子
        R_sou = np.matmul(center_sou, R)  # 使用旋转矩阵变换源点集
        s = np.sum(R_sou * center_tar) / np.sum(R_sou * R_sou)  # 计算比例因子
    else:
        s = 1  # 如果不需要缩放，比例因子为1

    # 计算平移向量
    t = np.mean(tar, axis=0) - s * np.matmul(np.mean(sou, axis=0), R)

    return R, t, s


In [6]:
def apply_transform(vertices,R,t,scale = 1.0):	
    pcl = [(np.array(v).reshape(1,-1).dot(scale).dot(R) + t).flatten().tolist() for v in vertices]
    return pcl

def apply_transform_2d(vertices, R, t, scale=1.0):
    # 对每个顶点应用变换
    pcl = [(np.array(v).reshape(1, -1).dot(scale * R) + t).flatten().tolist() for v in vertices]
    return pcl

In [4]:
# json.load(open(os.path.join(json_gt_path, str(i).zfill(6) + '.json'), 'r+'))[0]['keypoints3d']
source = json.load(open('/home/sparkle79/pypro/PoVER/swimXYZ/2D/free_wl_p3/vitprocess/000100.json','r+'))['annotations'][0]['keypoints']
target = json.load(open('/home/sparkle79/pypro/PoVER/swimXYZ/2D/free_wl_p3/gt/jsons/000100.json','r+'))['annotations'][0]['keypoints']

FileNotFoundError: [Errno 2] No such file or directory: './compare_jsons/jsons_mocap_without_ori/000001.json'

In [5]:
R,t,scale =  align_source_to_target(source,target,scale=True)
source = apply_transform(source,R,t,scale)

In [11]:
oripath = '/home/sparkle79/pypro/PoVER/swimXYZ/2D/free_wl_p3/vitprocess'
savepath = '/home/sparkle79/pypro/PoVER/swimXYZ/2D/free_wl_p3/vitpalign'
gtpath = '/home/sparkle79/pypro/PoVER/swimXYZ/2D/free_wl_p3/gt/jsons'
for i in range(100,161):

    source = json.load(open(os.path.join(oripath, str(i).zfill(6) + '.json'),'r+'))['annotations'][0]['keypoints']
    target = json.load(open(os.path.join(gtpath, str(i).zfill(6) + '.json'),'r+'))['annotations'][0]['keypoints']

    source2d = np.array(source).reshape(17,3)[:,:2]
    target2d = np.array(target).reshape(17,3)[:,:2]
    # print(source.shape)
    # print(target.shape)
    R,t,scale =  align_source_to_target_2d(source2d,target2d,scale=False)

    source1 = json.load(open(os.path.join(oripath, str(i).zfill(6) + '.json'),'r+'))

    sk2d = np.array(source1['annotations'][0]['keypoints']).reshape(17,3)[:,:2]

    pcl = apply_transform_2d(sk2d,R,t,scale)

    source3d = np.array(source).reshape(17,3)
    source3d[:,:2] = pcl

    source1['annotations'][0]['keypoints'] = source3d.reshape(1,-1).tolist()

    with open(os.path.join(savepath, str(i).zfill(6) + '.json'), "w+") as json_file:
        json.dump(source1, json_file, indent=4)